**WORKSHOP: SINGLE-CELL RNA-SEQ ANALYSIS WITH SCANPY**

**Objective:** We will process raw single-cell sequencing data from Rat (time-series radiation injury) and Human samples to identify cell types and visualize their response to injury.

**Part 1: Setup and Environment Configuration**

First, we import the necessary libraries. We use Scanpy for the core analysis, Pandas/NumPy for data manipulation, and Matplotlib/Seaborn for plotting. We also define our directory structure to ensure our results are saved systematically.

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import bbknn

sc.settings.verbosity = 3 
sc.set_figure_params(dpi=300, fontsize=8, facecolor="white", frameon=False, vector_friendly=True)

BASE_DIR = Path.cwd()
WRITE_DIR = BASE_DIR / "write"
RESULT_DIR = Path.home() / "results"

for d in [WRITE_DIR, RESULT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

for fig in ["1_preprocessing_and_clustering_Rat", "1_preprocessing_and_clustering_Human"]:
    (RESULT_DIR / fig).mkdir(exist_ok=True)

print(f"Results will be saved to: {RESULT_DIR}")

*sc.set_figure_params:* This sets global defaults for plots. dpi=300 ensures high resolution suitable for academic journals. vector_friendly=True ensures that when we save as PDF/SVG, the text remains editable.

*Path Management:* We use pathlib instead of standard strings. This makes the code robust, working seamlessly on both Windows (\) and Mac/Linux (/) file systems.

**Part 2: Data Loading Helper**

We need a robust function to load 10x Genomics data. A critical step here is "Standardization."

In [ ]:
def load_and_standardize(path, sample_id, condition, species):

    try:
        adata = sc.read_10x_mtx(BASE_DIR / path, var_names="gene_symbols", cache=False)
        adata.var_names_make_unique()
        adata.var_names = adata.var_names.str.upper() # Standardize for cross-species comparison
        adata.obs["sample_id"] = sample_id
        adata.obs["condition"] = condition
        adata.obs["species"] = species
        return adata
    except Exception as e:
        print(f"ERROR: Could not load {path} -> {e}")
        return None

*Standardization (.str.upper()):* Genes in mice/rats are often titled (e.g., Cd4, Krt14), while human genes are all caps (CD4, KRT14). To compare them later, we force everything to uppercase.

*AnnData Object:* This is Scanpy's core data structure. It holds the matrix (X), cell metadata (.obs), and gene metadata (.var).

**Part 3: The Processing Pipeline (QC, Normalization, Integration)**

This is the most critical section. We define a function process_basic that takes raw data and turns it into a clustered map.

In [ ]:
def process_basic(adata, batch_key):
 
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True)
    
    print(f"Number of cells before filtering: {adata.n_obs}")

    adata = adata[
        (adata.obs.total_counts > 800) & 
        (adata.obs.total_counts < 20000) & 
        (adata.obs.pct_counts_mt < 15) & 
        (adata.obs.n_genes_by_counts > 400) 
    ].copy()
    print(f"Number of cells after filtering: {adata.n_obs}")
    
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata) 
    adata.raw = adata  
    
    sc.pp.highly_variable_genes(adata, n_top_genes=3000, flavor="seurat_v3")
    
    sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])
    sc.pp.scale(adata)
    
    sc.pp.pca(adata, mask_var="highly_variable")

    bbknn.bbknn(adata, batch_key=batch_key)
  
    print("Calculating t-SNE...")
    sc.tl.tsne(adata, n_pcs=30) 
    
    print("Calculating UMAP...")
    sc.tl.umap(adata) 

    sc.tl.leiden(adata, resolution=2) 
    
    return adata

*1. QC (Quality Control):*
  
* *Mitochondrial Filtering:* When a cell membrane breaks (apoptosis/lysis), cytoplasmic RNA leaks out, but mitochondrial RNA (protected by double membranes) remains. High % mt (>15%) usually means the cell is dead or dying.

*2. Normalization (normalize_total):* We scale every cell to have 10,000 reads. This ensures we compare biology, not sequencing depth (e.g., Cell A shouldn't look like it has "more" genes just because the sequencer read it more times).

*3. Log Transformation (log1p):* Gene expression follows a power law. We take the log to make the data more normal (Gaussian), which is required for PCA and statistical testing.

*4. PCA (Principal Component Analysis):* Reduces the data from 20,000 genes to 50 "Principal Components" that capture the most variance (difference) in the dataset.

*5. BBKNN (Batch Balanced k-Nearest Neighbors):*

* *Problem:* If you sequence Control on Monday and Treated on Tuesday, they might cluster apart just due to "batch effects."

* *Solution:* BBKNN modifies the neighbor graph. For every cell, it forces the algorithm to find neighbors in each of the different batches. This "stitches" the datasets together without altering the actual gene expression values.

*6. Leiden Clustering:*

* This is a graph-based community detection algorithm. It improves upon the older "Louvain" algorithm by guaranteeing well-connected communities.

* *Resolution:* We use resolution=2. Higher resolution = more clusters (finer granularity). Lower resolution = fewer, broader clusters.

**Part 4: Automated Annotation Strategy**

Instead of manually checking every cluster, we score them against known biological markers.

In [ ]:
def auto_annotate_cell_types(adata, marker_dict):
 
    old_scores = [c for c in adata.obs.columns if c.startswith("score_")]
    if old_scores: adata.obs.drop(columns=old_scores, inplace=True)
    
    print("\n--- Marker Gene Check ---")
 
    for cell_type, genes in marker_dict.items():
        valid_genes = [g for g in genes if g in adata.var_names]
        if len(valid_genes) > 0:
            sc.tl.score_genes(adata, gene_list=valid_genes, score_name=f"score_{cell_type}")
        else:
            print(f"WARNING: NO marker genes found for {cell_type}!")
    
    score_cols = [col for col in adata.obs.columns if col.startswith("score_")]
    if not score_cols: return adata

    cluster_scores = adata.obs.groupby("leiden")[score_cols].mean()
    cluster_mapping = {}
    for cluster in cluster_scores.index:
        best_score_col = cluster_scores.loc[cluster].idxmax()
        cell_name = best_score_col.replace("score_", "")
        cluster_mapping[cluster] = cell_name
    
    adata.obs["cell_type"] = adata.obs["leiden"].map(cluster_mapping)
    return adata

*sc.tl.score_genes:* This calculates a score for a specific set of genes (e.g., T-Cell markers) for each cell. It compares the average expression of the marker set against a randomly selected set of reference genes to ensure the score is statistically robust.

*Logic:* If Cluster 5 has a high average score for "Keratinocytes" and a low score for everything else, we label it "Keratinocyte".

**Part 5: Visualization Helper**

We create a custom function to plot datasets side-by-side while keeping the axes locked.

In [ ]:
def plot_split_tsne(adata, keys, key_col, color_col, save_path, figsize_per_plot=(4, 4)):
  
    n_plots = len(keys)
    fig, axes = plt.subplots(1, n_plots, figsize=(figsize_per_plot[0] * n_plots, figsize_per_plot[1]))
    
    if n_plots == 1: axes = [axes] 

    x_min, x_max = adata.obsm['X_tsne'][:, 0].min(), adata.obsm['X_tsne'][:, 0].max()
    y_min, y_max = adata.obsm['X_tsne'][:, 1].min(), adata.obsm['X_tsne'][:, 1].max()
    pad_x = (x_max - x_min) * 0.05
    pad_y = (y_max - y_min) * 0.05

    for i, key in enumerate(keys):
        ax = axes[i]
        subset = adata[adata.obs[key_col] == key] 
        
        sc.pl.tsne(
            subset, 
            color=color_col, 
            ax=ax, 
            show=False, 
            title=key, 
            frameon=False,
            legend_loc="none" if i < n_plots - 1 else "right margin", 
            s=50 
        )
        
        ax.set_xlim(x_min - pad_x, x_max + pad_x)
        ax.set_ylim(y_min - pad_y, y_max + pad_y)
        
        ax.set_xlabel("tSNE_1")
        if i == 0: ax.set_ylabel("tSNE_2")
        else: ax.set_ylabel("")
            
    plt.tight_layout()
    plt.savefig(save_path, bbox_inches="tight")
    plt.close()
    print(f"Saved split t-SNE to: {save_path}")

*Why Split Plots?:* In dense datasets, points overlap. Plotting "Control" and "Treated" separately allows us to see if a specific cell population disappears or expands after treatment.

**Part 6: Defining Biological Markers**

We define the dictionary of genes that define our cell types.

In [ ]:
MARKERS_RAT = {
    "Keratinocytes (KC)": ["KRT1", "KRT14", "KRT5"],
    "Fibroblasts (FB)": ["DCN", "APOD", "CRABP1"],
    "Endothelial (EC)": ["TM4SF1", "CDH5", "CD93"],
    "Pericytes (PC)": ["RGS5", "DES", "PDGFRB"],
    "Schwann (SC)": ["GATM", "MPZ", "PLP1"],          
    "Smooth_Muscle (SMC)": ["MYH11", "MYL9", "TAGLN"],
    "Myoblasts (MB)": ["MYF5", "JSRP1", "CDH15"],
    "Neural (NC)": ["CMTM5", "BCHE", "AJAP1"],
    "Macrophages (Mø)": ["C1QC", "CD68", "PF4"],
    "Neutrophils (NEUT)": ["S100A8", "S100A9", "LYZ2"],
    "T_Cells (TC)": ["CD3D", "CD3E", "ICOS"],
    "B_Cells (BC)": ["TYROBP", "IFI30", "IGHM"],
    "Dendritic (DC)": ["CD207", "CD74", "RT1-DB1"]   
}

MARKERS_HUMAN = {
    "Keratinocytes (KC)": ["KRT14", "KRT5", "KRT1"],
    "Fibroblasts (FB)": ["DCN", "APOD", "COL1A1"],
    "Endothelial (EC)": ["CDH5", "CD93"],
    "Sweat_Gland (SGC)": ["DCD","AQP5"], 
    "Smooth_Muscle (SMC)": ["MYL9", "TAGLN", "ACTA2"],
    "Neural (NC)": ["CDH19", "SOX10", "PMP22"],
    "T_Cells (TC)": ["CD3D", "CD3E", "CD3G", "CCL5"],
    "NK_Cells (NK)": ["CD3D", "CD3E", "CD3G", "CCL5", "GZMB"],
    "Macrophages (Mø)": ["CD74", "AIF1", "CD68"],
    "Mast_Cells (MC)": ["TPSAB1", "TPSB2", "CPA3", "IL1RL1"],
    "Neutrophils (NEUT)": ["KRT14", "KRT5", "KRT1", "S100A8", "S100A9"]
}

**Part 7: Executing Rat Analysis**

We now run the pipeline on the Rat samples.

In [ ]:
print("\n--- FIGURE 1: RAT ANALYSIS ---")

rat_samples = [
    load_and_standardize("datas/rat/GSM5814220_con/", "Con", "Con", "rat"),
    load_and_standardize("datas/rat/GSM5814221_IR_7d/", "7d", "7d", "rat"),
    load_and_standardize("datas/rat/GSM5814222_IR_14d/", "14d", "14d", "rat"),
    load_and_standardize("datas/rat/GSM5814223_IR_28d/", "28d", "28d", "rat"),
]

adata_rat = sc.concat([r for r in rat_samples if r is not None], label="batch")
adata_rat = process_basic(adata_rat, batch_key="batch")
adata_rat = auto_annotate_cell_types(adata_rat, MARKERS_RAT)

sc.pl.tsne(adata_rat, color=["cell_type"], frameon=False, show=False, title="All Conditions Combined")
plt.savefig(RESULT_DIR / "1_preprocessing_and_clustering_Rat/tSNE_Combined.png", bbox_inches="tight")
plt.close()



![alt text](result1/Fig1_tSNE_Combined.png)

In [ ]:
plot_split_tsne(
    adata_rat, 
    keys=["Con", "7d", "14d", "28d"], 
    key_col="condition", 
    color_col="cell_type", 
    save_path=RESULT_DIR / "1_preprocessing_and_clustering_Rat/tSNE_Split.png"
)

![alt text](result1/Fig1D_tSNE_Split.png)

In [ ]:
all_rat_markers = [g for list in MARKERS_RAT.values() for g in list]
all_rat_markers = list(dict.fromkeys(all_rat_markers))
sc.pl.dotplot(adata_rat, all_rat_markers, groupby="cell_type", standard_scale="var", show=False)
plt.savefig(RESULT_DIR / "1_preprocessing_and_clustering_Rat/Dotplot.png", bbox_inches="tight")
plt.close()

![alt text](result1/Fig1C_Dotplot.png)

In [ ]:
counts = adata_rat.obs.groupby(["condition", "cell_type"]).size().unstack(fill_value=0)
props = counts.div(counts.sum(axis=1), axis=0) * 100
props = props.loc[["Con", "7d", "14d", "28d"]]
props.plot(kind="bar", stacked=True, figsize=(8, 5), colormap="tab20")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("Cell Type Proportions")
plt.tight_layout()
plt.savefig(RESULT_DIR / "1_preprocessing_and_clustering_Rat/Proportions.png")
plt.close()

![alt text](result1/Fig1E_Proportions.png)

*t-SNE:* Represents cells as points in 2D space. Cells that are biologically similar (like T-cells) will group together.

*Dot Plot:*

* *X-axis:* Genes. Y-axis: Cell Types.

* *Dot Size:* Percentage of cells in that group expressing the gene.

* *Color:* Average expression level.

* *Purpose:* This validates our annotation. If the "T-Cell" cluster doesn't light up for CD3D, our clustering or annotation is wrong.

*Stacked Bar Chart:* Allows us to answer "Does the proportion of Fibroblasts increase 14 days after radiation?"

**Part 8: Executing Human Analysis**

We process the Human samples for comparison.

In [ ]:
print("\n--- FIGURE 2: HUMAN ANALYSIS ---")

human_samples = [
    load_and_standardize("datas/human/GSM5821748_con/", "Con", "Con", "human"),
    load_and_standardize("datas/human/GSM5821749_IR/", "IR", "IR", "human"),
]

adata_human = sc.concat([h for h in human_samples if h is not None], label="batch")
adata_human = process_basic(adata_human, batch_key="batch")
adata_human = auto_annotate_cell_types(adata_human, MARKERS_HUMAN)

sc.pl.tsne(adata_human, color=["cell_type"], frameon=False, show=False, title="All Conditions Combined")
plt.savefig(RESULT_DIR / "1_preprocessing_and_clustering_Human/tSNE_Combined.png", bbox_inches="tight")
plt.close()

![alt text](result1/Fig2_tSNE_Combined.png)

In [ ]:
plot_split_tsne(
    adata_human, 
    keys=["Con", "IR"], 
    key_col="condition", 
    color_col="cell_type", 
    save_path=RESULT_DIR / "1_preprocessing_and_clustering_Human/tSNE_Split.png"
)

![alt text](result1/Fig2_tSNE_Split.png)

In [ ]:
all_human_markers = [g for list in MARKERS_HUMAN.values() for g in list]
all_human_markers.append("NR4A1") 
all_human_markers = list(dict.fromkeys(all_human_markers))
sc.pl.dotplot(adata_human, all_human_markers, groupby="cell_type", standard_scale="var", show=False)
plt.savefig(RESULT_DIR / "1_preprocessing_and_clustering_Human/Dotplot.png", bbox_inches="tight")
plt.close()

![alt text](result1/Fig2D_Dotplot.png)

In [ ]:
counts_human = adata_human.obs.groupby(["condition", "cell_type"]).size().unstack(fill_value=0)
props_human = counts_human.div(counts_human.sum(axis=1), axis=0) * 100
props_human = props_human.loc[["Con", "IR"]] 
props_human.plot(kind="bar", stacked=True, figsize=(6, 5), colormap="tab20")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title("Human Cell Type Proportions")
plt.tight_layout()
plt.savefig(RESULT_DIR / "1_preprocessing_and_clustering_Human/Proportions.png")
plt.close()

![alt text](result1/Fig2E_Proportions.png)

While this workflow uses a pre-defined marker list to name clusters, in a discovery phase (where you don't know the cell types), you would use the Wilcoxon Rank-Sum Test.

* *Function:* sc.tl.rank_genes_groups(adata, method='wilcoxon')

* *Purpose:* It statistically compares the expression of every gene in Cluster 0 vs. all other clusters.

* *Why Wilcoxon?:* Gene expression is not normally distributed (it has many zeros and heavy tails). The t-test assumes a normal distribution and fails on single-cell data. Wilcoxon is a non-parametric test (it ranks values instead of using raw values), making it the gold standard for finding marker genes in scRNA-seq.

**Crucial Step for Continuing the Workshop:** If you intend to execute the subsequent code blocks (for Figure 3), you must first strictly isolate the Fibroblast population using the code below. By extracting these cells and reverting to the .raw data, we remove the global influence of other cell types (like T-cells or Keratinocytes) on the gene expression statistics. This is a mandatory prerequisite that allows us to re-cluster and analyze heterogeneity specifically within the fibroblast lineage in the next section."

In [ ]:
#Add this code at the bottom of the Rat Analysis segment (Part 7).
print("   > Extracting Rat Fibroblasts...")
rat_fibro = adata_rat[adata_rat.obs["cell_type"].str.contains("Fibro", na=False)].copy()
if rat_fibro.raw is not None: rat_fibro = rat_fibro.raw.to_adata()

#Add this code at the bottom of the Executing Human Analysis segment (Part 8).
print("   > Extracting Human Fibroblasts...")
human_fibro = adata_human[adata_human.obs["cell_type"].str.contains("Fibro", na=False)].copy()
if human_fibro.raw is not None: human_fibro = human_fibro.raw.to_adata()

In Figure 3, we ran sc.pp.scale(), which centers gene expression to mean=0 and variance=1. This scaling was based on the entire dataset. 

When we zoom in on Fibroblasts:

* The variance between cell types is no longer relevant.

* We want to find subtle variance within the fibroblast population (e.g., myofibroblasts vs. inflammatory fibroblasts).

* Therefore, we revert to .raw (unscaled data) so we can calculate new Highly Variable Genes (HVGs) specific only to fibroblasts.

**Part 9: Advanced Sub-clustering Strategy**

When zooming in on a specific cell type, standard clustering often fails to capture subtle differences. We use an Automated Resolution Tuning approach to find the perfect granularity.

In [ ]:
print("\n--- FIGURE 3: FIBROBLAST DETAILED ANALYSIS ---")

(RESULT_DIR / "2_trajectory_analysis").mkdir(parents=True, exist_ok=True)

def auto_tune_resolution(adata, target_clusters):
    res_low, res_high = 0.1, 2.0
    for _ in range(15):
        current_res = (res_low + res_high) / 2
        sc.tl.leiden(adata, resolution=current_res, key_added="sub_leiden")
        n_clusters = len(adata.obs['sub_leiden'].unique())
        
        if n_clusters == target_clusters: return current_res
        if n_clusters > target_clusters: res_high = current_res 
        else: res_low = current_res 
    return current_res

*The Problem:* The resolution parameter in Leiden clustering is arbitrary. resolution=0.5 might give 3 clusters, while 1.0 gives 8.

*The Solution (Binary Search):* Instead of guessing, this function mathematically "hunts" for the specific resolution that yields exactly the number of clusters we expect biologically (e.g., if we know there are roughly 5 fibroblast subtypes in humans).

**Part 10: Trajectory Inference (PAGA & ForceAtlas2)**

This is the most advanced part of the pipeline. We reconstruct the "developmental tree" of the cells.

In [ ]:
def analyze_fibroblast_subtypes_branched(adata, species_name):
    
    print(f"   > Processing {species_name} Fibroblasts...")
    
    sc.pp.filter_genes(adata, min_cells=3)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata 
    
    sc.pp.highly_variable_genes(adata, n_top_genes=1500, flavor='seurat')
    sc.pp.scale(adata)
    sc.pp.pca(adata)

    sc.pp.neighbors(adata, n_neighbors=10, use_rep="X_pca") 
    
    target = 7 if species_name == "Rat" else 5
    res = auto_tune_resolution(adata, target)
    sc.tl.leiden(adata, resolution=res, key_added="sub_leiden")
    
    cluster_map = {str(c): f"FB{i+1}" for i, c in enumerate(sorted(adata.obs['sub_leiden'].unique().astype(int)))}
    adata.obs["sub_type"] = adata.obs["sub_leiden"].map(cluster_map)
    
    sc.tl.paga(adata, groups="sub_type")
    sc.pl.paga(adata, plot=False) 
    
    print(f"     > Calculating branched tree structure...")
    layout_key = 'fr' 
    if 'fa2' in locals() or 'fa2' in globals():
        try:
            sc.tl.draw_graph(adata, init_pos='paga', layout='fa', random_state=42)
            layout_key = 'fa'
        except:
            sc.tl.draw_graph(adata, init_pos='paga', layout='fr')
    else:
        sc.tl.draw_graph(adata, init_pos='paga', layout='fr')

    con_counts = adata.obs.groupby("sub_type")["condition"].apply(lambda x: (x == "Con").sum())
    root_clust = con_counts.idxmax()
    
    root_cells = np.flatnonzero(adata.obs["sub_type"] == root_clust)
    adata.uns["iroot"] = root_cells[0] if len(root_cells) > 0 else 0 
    
    sc.tl.diffmap(adata)
    sc.tl.dpt(adata)
    
    sc.tl.tsne(adata, n_pcs=20)
    
    return adata

rat_fibro = analyze_fibroblast_subtypes_branched(rat_fibro, "Rat")
human_fibro = analyze_fibroblast_subtypes_branched(human_fibro, "Human")

*1. Re-calculation:* We re-run PCA and Neighbors because the variable genes that distinguish Fibroblast A from Fibroblast B are different from those that distinguish Fibroblasts from T-Cells.

*2. PAGA (Partition-based Graph Abstraction):*

* Standard t-SNE/UMAP often breaks continuous trajectories into separate islands.

* PAGA draws a "map of connectivity" between clusters. It tells us: "Cluster 1 is strongly connected to Cluster 2, but not Cluster 3."

*3. ForceAtlas2 (FA2):*

* This is a graph layout algorithm that simulates physics. Nodes (cells) repel each other like magnets, but edges (similarities) pull them together like springs.

* We initialize FA2 using the PAGA positions (init_pos='paga'). This forces the single-cell plot to respect the biological topology, revealing tree-like differentiation paths.

*4. Diffusion Pseudotime (DPT):*

* We cannot measure "Real Time" in a single-cell experiment because the cell is destroyed when sequenced.

* Pseudotime is a mathematical proxy. We set a "Start Cell" (Root, usually from the Control group). DPT measures the probability distance of every other cell from that root.

* *Result:* A scale from 0 (stem-like/naive) to 1 (terminally differentiated/damaged).

**Part 11: Transcription Factor Enrichment**

We visualize which Transcription Factors (TFs) drive the changes observed in the trajectory.

In [ ]:

print("\n--- Generating Figure 3 Plots ---")

RAT_TFS = ["BACH1", "ETS1", "NFKB1", "RFX5", "SMAD1", "STAT2", "ATF3", "FOS", "FOSB", 
"JUN", "JUND", "IRF7", "IRF1", "CEBPE", "HOXD4", "DLX5", "GRHL2", "NFE2L1", "POU3F1", "SREBF2"]

HUMAN_TFS = ["EHF", "EMX2", "FOXA1", "HOXB3", "HOXB4", "HOXC6", "HOXC8", "HOXC9", 
"IRF6", "MESP1", "FOSL1", "NFE2L2", "NR2F1", "RARB", "SREBF2", "TEAD4", "MAFA", "NFATC1", "LHX9", "ETV3"]

def plot_fig3a(adata, genes, filename, order):
    valid = [g for g in genes if g in adata.raw.var_names]
    present = [c for c in order if c in adata.obs["condition"].unique()]
    
    adata.obs["condition"] = adata.obs["condition"].astype("category").cat.reorder_categories(present)
    
    if valid:
        sc.pl.dotplot(adata, valid, groupby="condition", standard_scale="var", show=False, title="TF Enrichment")
        plt.savefig(RESULT_DIR / f"2_trajectory_analysis/{filename}", bbox_inches="tight"); plt.close()

plot_fig3a(rat_fibro, RAT_TFS, "Rat_TF.png", ["Con", "7d", "14d", "28d"])
plot_fig3a(human_fibro, HUMAN_TFS, "Human_TF.png", ["Con", "IR"])

![alt text](result1/Fig3A_Rat_TF.png) ![alt text](result1/Fig3A_Human_TF.png)

**Part 12: Differential Expression & Heterogeneity**

Here we visualize the sub-clusters and validate specific genes of interest (like NUR77/NR4A1).

In [ ]:
COMMON_DEGS = ["NUR77", "NR4A1", "CDKN1A", "DPT", "S100A10", "PROCR"]

valid_rat = [g for g in COMMON_DEGS if g in rat_fibro.raw.var_names]
if valid_rat:
    sc.pl.violin(rat_fibro, valid_rat, groupby="condition", rotation=90, show=False, use_raw=True)
    plt.savefig(RESULT_DIR / "2_trajectory_analysis/Rat_CommonDEGs.png", bbox_inches="tight"); plt.close()

    valid_human = [g for g in COMMON_DEGS if g in human_fibro.raw.var_names]
if valid_human:
    sc.pl.violin(human_fibro, valid_human, groupby="condition", rotation=90, show=False, use_raw=True)
    plt.savefig(RESULT_DIR / "2_trajectory_analysis/Human_CommonDEGs.png", bbox_inches="tight"); plt.close()

![alt text](result1/Fig3B_Rat_CommonDEGs.png) ![alt text](result1/Fig3B_Human_CommonDEGs.png)

In [ ]:
tsne_rat = sc.pl.tsne(rat_fibro, color="sub_type", legend_loc="on data", palette="tab10", 
                      title="Rat Fibroblasts", return_fig=True, show=False)
tsne_rat.savefig(RESULT_DIR / "2_trajectory_analysis/Rat_tSNE.png", bbox_inches="tight")
plt.close(tsne_rat)

plot_split_tsne(rat_fibro, keys=["Con", "7d", "14d", "28d"], key_col="condition", color_col="sub_type", 
                save_path=RESULT_DIR / "2_trajectory_analysis/Rat_tSNE_Split.png")

![alt text](result1/Fig3C_Rat_tSNE.png) ![alt text](result1/Fig3C_Rat_tSNE_Split.png)

In [ ]:
tsne_human = sc.pl.tsne(human_fibro, color="sub_type", legend_loc="on data", palette="tab10", 
                        title="Human Fibroblasts", return_fig=True, show=False)
tsne_human.savefig(RESULT_DIR / "2_trajectory_analysis/Human_tSNE.png", bbox_inches="tight")
plt.close(tsne_human)

plot_split_tsne(human_fibro, keys=["Con", "IR"], key_col="condition", color_col="sub_type", 
                save_path=RESULT_DIR / "2_trajectory_analysis/Human_tSNE_Split.png")

![alt text](result1/Fig3C_Human_tSNE.png) ![alt text](result1/Fig3C_Human_tSNE_Split.png)

In [ ]:
def plot_nur77(adata, name):
    gene = "NR4A1" if "NR4A1" in adata.raw.var_names else "NUR77"
    if gene in adata.raw.var_names:
        sc.pl.violin(adata, gene, groupby="sub_type", rotation=0, use_raw=True, palette="tab10", show=False)
        plt.title(f"{name} Nur77 Expression")
        plt.savefig(RESULT_DIR / f"2_trajectory_analysis/{name}_Nur77.png", bbox_inches="tight"); plt.close()

plot_nur77(rat_fibro, "Rat")
plot_nur77(human_fibro, "Human")

![alt text](result1/Fig3D_Rat_Nur77.png) ![alt text](result1/Fig3D_Human_Nur77.png)

**Part 13: Visualizing the Trajectory Tree**

Finally, we generate the PAGA-initialized ForceAtlas2 plots. These are the complex "tree" structures often seen in high-impact journals.

In [ ]:

def plot_trajectory_branched(adata, name, label):
    layout_key = 'fa' if 'X_draw_graph_fa' in adata.obsm else 'fr'
    print(f"   > Plotting {name} branched trajectory using '{layout_key}'...")
    
    gene = "NR4A1" if "NR4A1" in adata.raw.var_names else "NUR77"

    sc.pl.draw_graph(adata, color="dpt_pseudotime", layout=layout_key, 
                      frameon=False, show=False, cmap="viridis", 
                      edges=True, edges_width=0.2, 
                      title=f"{name} Pseudotime (Trajectory)")
    plt.savefig(RESULT_DIR / f"2_trajectory_analysis/{label}_{name}_Pseudotime_Trajectory.png", bbox_inches="tight")
    plt.close()

![alt text](result1/Fig3E_Rat_Pseudotime_Trajectory.png) ![alt text](result1/Fig3F_Human_Pseudotime_Trajectory.png)

In [ ]:
if gene in adata.raw.var_names:
        raw_data = adata.raw[:, gene].X
        if hasattr(raw_data, "toarray"): raw_data = raw_data.toarray()
        raw_data = raw_data.flatten()
        
        counts = np.expm1(raw_data)
        custom_val = np.log10(counts + 0.1)
        
        plot_col_name = f"{gene}_log10"
        adata.obs[plot_col_name] = custom_val

        sc.pl.draw_graph(adata, color=plot_col_name, layout=layout_key, 
                         frameon=False, show=False, cmap="viridis", 
                         edges=True, edges_width=0.2, 
                         title=f"{name} {gene} (log10(val+0.1))")
        plt.savefig(RESULT_DIR / f"2_trajectory_analysis/{label}_{name}_{gene}_Trajectory.png", bbox_inches="tight")
        plt.close()

![alt text](result1/Fig3E_Rat_NR4A1_Trajectory.png) ![alt text](result1/Fig3F_Human_NR4A1_Trajectory.png)

In [ ]:
 plt.figure(figsize=(6, 4))
        sc.pl.scatter(adata, x='dpt_pseudotime', y=plot_col_name, color='sub_type', 
                      show=False, title=f"{name} {gene} across Pseudotime")
        plt.xlabel("Pseudotime (Start -> End)")
        plt.ylabel(f"{gene} Expression (log10(val+0.1))")
        plt.savefig(RESULT_DIR / f"2_trajectory_analysis/{label}_{name}_{gene}_vs_Pseudotime_Scatter.png", bbox_inches="tight")
        plt.close()

![alt text](result1/Fig3E_Rat_NR4A1_vs_Pseudotime_Scatter.png) ![alt text](result1/Fig3F_Human_NR4A1_vs_Pseudotime_Scatter.png)

In [ ]:
 sc.pl.draw_graph(adata, color="sub_type", layout=layout_key, 
                      frameon=False, show=False, palette="tab10", 
                      edges=True, edges_width=0.3, 
                      title=f"{name} Subtype Branches")
    plt.savefig(RESULT_DIR / f"2_trajectory_analysis/{label}_{name}_Subtypes_Trajectory.png", bbox_inches="tight"); plt.close()

![alt text](result1/Fig3E_Rat_Subtypes_Trajectory.png) ![alt text](result1/Fig3F_Human_Subtypes_Trajectory.png)

In [ ]:
if name == "Rat":
        cond_order = ["Con", "7d", "14d", "28d"]
    else:
        cond_order = ["Con", "IR"]
    
    basis_key = f"X_draw_graph_{layout_key}"
    x_min, x_max = adata.obsm[basis_key][:, 0].min(), adata.obsm[basis_key][:, 0].max()
    y_min, y_max = adata.obsm[basis_key][:, 1].min(), adata.obsm[basis_key][:, 1].max()
    pad_x = (x_max - x_min) * 0.05
    pad_y = (y_max - y_min) * 0.05

    n_conds = len(cond_order)
    fig, axes = plt.subplots(1, n_conds, figsize=(4 * n_conds, 4))
    if n_conds == 1: axes = [axes]

    for i, cond in enumerate(cond_order):
        ax = axes[i]
        subset = adata[adata.obs["condition"] == cond]
        
        sc.pl.draw_graph(
            subset, 
            color="condition", 
            layout=layout_key, 
            ax=ax, 
            show=False, 
            title=cond, 
            frameon=False, 
            legend_loc="none", 
            s=50
        )
        
        ax.set_xlim(x_min - pad_x, x_max + pad_x)
        ax.set_ylim(y_min - pad_y, y_max + pad_y)
        ax.set_xlabel("")
        ax.set_ylabel("")

    plt.tight_layout()
    plt.savefig(RESULT_DIR / f"2_trajectory_analysis/{label}_{name}_Condition_Split.png", bbox_inches="tight")
    plt.close()

plot_trajectory_branched(rat_fibro, "Rat", "E")
plot_trajectory_branched(human_fibro, "Human", "F")

print("\n--- ALL ANALYSES COMPLETE ---")

![alt text](result1/Fig3E_Rat_Condition_Split.png) ![alt text](result1/Fig3F_Human_Condition_Split.png)

*Draw Graph (sc.pl.draw_graph):* This visualizes the PAGA/ForceAtlas2 embedding. Unlike UMAP (which separates clusters), this layout tries to keep continuous lineages connected.

*Pseudotime vs Expression Scatter:*

* X-axis: Time (0.0 to 1.0).

* Y-axis: Gene Expression.

* Interpretation: If you see a curve going UP as X increases, that gene is upregulated during the injury response.

*Condition Split:* Similar to the t-SNE split in Figure 1, this shows us which parts of the "tree" are occupied by healthy cells vs. irradiated cells. Usually, "Control" cells sit at the root, and "14d/28d" cells extend into the tips of the branches.